# [570. Managers with at Least 5 Direct Reports](https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Employee

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
| department  | varchar |
| managerId   | int     |
+-------------+---------+</pre>
id is the primary key (column with unique values) for this table.
Each row of this table indicates the name of an employee, their department, and the id of their manager.
If managerId is null, then the employee does not have a manager.
No employee will be the manager of themself.
 

Write a solution to find managers with at least five direct reports.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
<pre>+-----+-------+------------+-----------+
| id  | name  | department | managerId |
+-----+-------+------------+-----------+
| 101 | John  | A          | None      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |
+-----+-------+------------+-----------+</pre>
Output: 
<pre>+------+
| name |
+------+
| John |
+------+</pre>

In [1]:
# Pandas schema

import pandas as pd

data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

# to pyspark schema

#Importing the packages
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *



schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("managerId", IntegerType(), True)
])


spark = SparkSession.builder.getOrCreate()

employee_df = spark.createDataFrame(data,schema=schema)
employee_df.show(truncate=False)

+---+-----+----------+---------+
|id |name |department|managerId|
+---+-----+----------+---------+
|101|John |A         |NULL     |
|102|Dan  |A         |101      |
|103|James|A         |101      |
|104|Amy  |A         |101      |
|105|Anne |A         |101      |
|106|Ron  |B         |101      |
+---+-----+----------+---------+



In [2]:
# in Spark Dataframe
employee_df.alias('e1').join(employee_df.alias('e2'),col('e1.id')==col('e2.managerId'),'inner')\
    .groupBy('e1.id','e1.name').agg(count('*').alias("count"))\
    .filter('count > 4')\
    .select('e1.name')\
    .show()


+----+
|name|
+----+
|John|
+----+



In [3]:
# in Spark SQL
employee_df.createOrReplaceTempView('Employee')
spark.sql('''
SELECT E1.NAME FROM 
EMPLOYEE E1 JOIN EMPLOYEE E2
ON
E1.ID=E2.MANAGERID
GROUP BY E1.ID,E1.NAME
HAVING COUNT(*)>4
''').show()

+----+
|NAME|
+----+
|John|
+----+



In [4]:
spark.stop()